In [4]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

In [5]:
def read_folder_npy_data(folder_path):
    data_dict = {}

    # Filter only .npy files
    npy_files = [f for f in os.listdir(folder_path) if f.endswith('.npy')]

    # Load .npy files
    for filename in npy_files:
        file_path = os.path.join(folder_path, filename)
        
        try:
            # Attempt to load the file
            content = np.load(file_path, allow_pickle=True)

            # Print information about the loaded file
            print(f"Loaded {filename}: Shape={content.shape}, Dtype={content.dtype}")
            
            # Add to the dictionary
            data_dict[filename] = content
        
        except Exception as e:
            # Print an error message if loading fails
            print(f"Error loading {filename}: {e}")

    return data_dict



In [6]:
lyh_data_path = "Emotiv_dataloader-main/data"
data_dict = read_folder_npy_data(lyh_data_path)

Loaded right2_orginal.npy: Shape=(100,), Dtype=object
Loaded left1_orginal.npy: Shape=(100,), Dtype=object
Loaded nothing2_orginal.npy: Shape=(100,), Dtype=object
Loaded left3_orginal.npy: Shape=(100,), Dtype=object
Loaded right3_orginal.npy: Shape=(100,), Dtype=object
Loaded leg2_orginal.npy: Shape=(100,), Dtype=object
Loaded right_processed.npy: Shape=(15, 300000), Dtype=float64
Loaded left2_orginal.npy: Shape=(100,), Dtype=object
Loaded left_processed.npy: Shape=(15, 300000), Dtype=float64
Loaded nothing3_orginal.npy: Shape=(100,), Dtype=object
Loaded leg1_orginal.npy: Shape=(100,), Dtype=object
Loaded nothing1_orginal.npy: Shape=(100,), Dtype=object
Loaded nothing_processed.npy: Shape=(15, 300000), Dtype=float64
Loaded leg3_orginal.npy: Shape=(100,), Dtype=object
Loaded leg_processed.npy: Shape=(15, 300000), Dtype=float64
Loaded right1_orginal.npy: Shape=(100,), Dtype=object


In [7]:
for file in sorted(data_dict.keys()):
    print("file: {:<30} data shape: {}".format(file, data_dict[file].shape))

file: left1_orginal.npy              data shape: (100,)
file: left2_orginal.npy              data shape: (100,)
file: left3_orginal.npy              data shape: (100,)
file: left_processed.npy             data shape: (15, 300000)
file: leg1_orginal.npy               data shape: (100,)
file: leg2_orginal.npy               data shape: (100,)
file: leg3_orginal.npy               data shape: (100,)
file: leg_processed.npy              data shape: (15, 300000)
file: nothing1_orginal.npy           data shape: (100,)
file: nothing2_orginal.npy           data shape: (100,)
file: nothing3_orginal.npy           data shape: (100,)
file: nothing_processed.npy          data shape: (15, 300000)
file: right1_orginal.npy             data shape: (100,)
file: right2_orginal.npy             data shape: (100,)
file: right3_orginal.npy             data shape: (100,)
file: right_processed.npy            data shape: (15, 300000)


In [29]:
keys = ["nothing_processed.npy", "left_processed.npy", "right_processed.npy", "leg_processed.npy"]
train_data, train_labels, test_data, test_labels = [], [], [], []
for key in keys:
    d = [data_dict[key][0:14, i:i+1000] for i in range(0, data_dict[key].shape[1], 1000)]
    train_data += d[:int(len(d) * 0.8)]
    train_labels += [keys.index(key)] * len(d[:int(len(d) * 0.8)])
    test_data += d[int(len(d) * 0.8):]
    test_labels += [keys.index(key)] * len(d[int(len(d) * 0.8):])


In [ ]:
left_data = [data_dict["left_processed.npy"][0:14, i:i+1000] for i in range(0, data_dict["left_processed.npy"].shape[1], 1000)]

In [9]:
left_label = [1] * len(left_data)

In [10]:
left_data = torch.Tensor(left_data)

In [11]:
right_data = [data_dict["right_processed.npy"][0:14, i:i+1000] \
    for i in range(0, data_dict["right_processed.npy"].shape[1], 1000)]
right_label = [0] * len(right_data)
right_data = torch.Tensor(right_data)

In [41]:
def draw_save(data, label_list, label_names, ch_names, save_path):
    '''
    data: (n, n_channels, n_times)
    '''
    # 这里对提取的数据进行绘制时频图并保存
    # label_list = [str(i) for i in label_list]
    ch_len = len(ch_names)
    label_len = len(label_list)
    for count in range(label_len):
        # epochs标签(data.shape[0])
        for i in range(ch_len):
            # 通道数
            plt.rcParams['figure.figsize'] = (4.48, 4.48)  # 2.24, 2.24 设置figure_size尺寸
            plt.rcParams['savefig.dpi'] = 50  # 图片像素 这样输出的就是4.48*50=224
            plt.specgram(data[count][i], NFFT=16, Fs=128, noverlap=10)
            q = str(count)
            # path = save_path + label_list[count] + "/" + ch_names[i] + "_" + k + "_" + q + ".png"
            path = save_path + label_names[label_list[count]] + "/"
            if not os.path.exists(path):
                os.mkdir(path)
            path += ch_names[i] + "_" + q + ".png"
            plt.savefig(path, bbox_inches='tight', pad_inches=0)  # 后两项为去除白边
            # print("1")
 
    print(save_path + "has been finished\n")

In [42]:
channel_names = ["AF3","F7","F3","FC5","T7","P7","O1","O2","P8","T8","FC6","F4","F8","AF4"]
label_names = ["nothing", "left", "right", "leg"]
figure_data_path = os.getcwd() + "/figure_like_data/"
if not os.path.exists(figure_data_path):
    os.mkdir(figure_data_path)
figure_data_path_train = figure_data_path + "train/"
if not os.path.exists(figure_data_path_train):
    os.mkdir(figure_data_path_train)
figure_data_path_test = figure_data_path + "test/"
if not os.path.exists(figure_data_path_test):
    os.mkdir(figure_data_path_test)
draw_save(train_data, train_labels, label_names, channel_names, figure_data_path_train)